In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import regex
import json
import pandas as pd
import datasets

from tqdm.auto import tqdm
from grapheme import graphemes

/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# def extract_sanskrit(text: str) -> str:
#     """
#     입력 문자열에서 데바나가리 문자(산스크리트)를 추출하고
#     나머지 기호/영문/숫자 등은 제거한 뒤 반환합니다.
#     """
#     # \p{Devanagari}는 유니코드 스크립트 기반 정규식 (데바나가리 블록 전체)
#     tokens = regex.findall(r'\p{Devanagari}+', text)
#     return " ".join(tokens)

In [ ]:
# import regex

# # 기본 정의
# CONSONANT        = r'[\u0915-\u0939]'                    # क-ह
# VIRAMA           = r'\u094D'                             # ् (halant)
# VOWEL_SIGN       = r'[\u093E-\u094C\u0962\u0963]'        # ा-ौ, ॢ ॣ
# INDEPENDENT_VOWEL= r'[\u0904-\u0914]'                    # ऄ-औ (독립 모음)
# DIACRITICS       = r'[\u0900-\u0903]'                    # ऀ, ँ, ं, ः
# ZWJ_ZWNJ         = r'[\u200C\u200D]'                     # 합자 조절 ZWJ/ZWNJ

# # 자음군 (conjunct consonants)
# CONJUNCT = fr'{CONSONANT}(?:{VIRAMA}{ZWJ_ZWNJ}?{CONSONANT})*'

# # 개선된 'अक्षर' 단위 정규식
# AKSHARA_PATTERN = regex.compile(
#     fr'(?:{INDEPENDENT_VOWEL}|{CONJUNCT}(?:{VOWEL_SIGN})?)(?:{DIACRITICS})?'
# )

# def is_valid_sanskrit_word(word: str) -> bool:
#     """
#     산스크리트 문자열이 정상적인 '글자 단위(अक्षर, akṣara)'로만 구성되었는지 판별.
#     - 독립 모음 (अ, आ, इ, ...)
#     - 자음 단독, 자음 + 모음기호
#     - 자음 결합 (halant/ZWJ/ZWNJ로 연결)
#     - 발음 보조 기호 (अनुस्वार, विसर्ग, चन्द्रबिन्दु 등)
#     - halant 종결도 허용 (क् 등)
#     """
#     tokens = regex.findall(r'\X', word)  # grapheme cluster 단위 분리
    
#     for t in tokens:
#         if not AKSHARA_PATTERN.fullmatch(t):
#             return False
#     return True


In [ ]:
# def preproc_df(df):
#     sentences = []
#     sentences_noisy = []
#     for idx, row in tqdm(df.iterrows(), total=len(df)):
#         sentence_noisy = row['input_text'].strip()
#         sentence = row['target_text'].strip()
#         sentence_noisy_chunks = sentence_noisy.split('.')
#         sentence_chunks = sentence.split('.')
#         sentence_noisy_chunks = [sentence_noisy_chunk.strip() for sentence_noisy_chunk in sentence_noisy_chunks]
#         sentence_chunks = [sentence_chunk.strip() for sentence_chunk in sentence_chunks]
#         if len(sentence_noisy_chunks) != len(sentence_chunks):
#             continue
#         sentence_chunks_result = []
#         sentence_noisy_chunks_result = []
#         for sentence_noisy_chunk, sentence_chunk in zip(sentence_noisy_chunks, sentence_chunks):
#             # sentence_noisy_chunk = extract_sanskrit(sentence_noisy_chunk.strip())
#             sentence_chunk = extract_sanskrit(sentence_chunk.strip())
#             sentence_noisy_chunks_result.append(sentence_noisy_chunk)
#             sentence_chunks_result.append(sentence_chunk)
#         sentence_noisy = '. '.join(sentence_noisy_chunks_result).strip()
#         sentence = '. '.join(sentence_chunks_result).strip()
        
#         sentence_noisy_len = len(list(graphemes(sentence_noisy)))
#         sentence_len = len(list(graphemes(sentence)))

#         if sentence_len != sentence_noisy_len:
#             continue

#         break_flag = False
#         for word in sentence.split(' '):
#             if not is_valid_sanskrit_word(word):
#                 break_flag = True
#         if break_flag:
#             continue

#         sentences.append(sentence)
#         sentences_noisy.append(sentence_noisy)
#     return sentences, sentences_noisy


In [22]:
def char_max_check(sentence, n=3):
    for char in graphemes(sentence):
        if len(char)>n:
            return False
    return True

In [23]:
def preproc_df(df):
    sentences = []
    sentences_noisy = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        sentence_noisy = row['input_text'].strip()
        sentence = row['target_text'].strip()
        sentence_noisy_len = len(list(graphemes(sentence_noisy)))
        sentence_len = len(list(graphemes(sentence)))
        if sentence_noisy_len!=sentence_len:
            continue
        if not char_max_check(sentence):
            continue

        sentences.append(sentence)
        sentences_noisy.append(sentence_noisy)
    return sentences, sentences_noisy


In [24]:
train_df = pd.read_csv('PeOcrSanskrit/train_devnagari.csv')
dev_df = pd.read_csv('PeOcrSanskrit/val_devnagari.csv')
test_df = pd.read_csv('PeOcrSanskrit/test_devnagari.csv')

In [25]:
sentences, sentences_noisy = preproc_df(train_df)
train_df = pd.DataFrame({
    'sentence': sentences,
    'sentence_noisy': sentences_noisy,
})  
train_ds = datasets.Dataset.from_pandas(train_df)

100%|██████████| 208137/208137 [00:41<00:00, 5019.56it/s]


In [12]:
(train_df['input_text'] == train_df['target_text']).mean()

np.float64(0.026987032579502922)

In [27]:
(train_df['sentence'] == train_df['sentence_noisy']).mean()

np.float64(0.5695332739637305)

In [26]:
train_df

,sentence,sentence_noisy
0,स्वमन्यथा हल्यइत वर्ग जननाशं करोति च ।,स्वमन्यथा हल्यइत वर्ग जननाशं करोति च ।
1,कक्षायां प्रत्यहं याति समसूत्रानुसारतः ।,कक्षायां प्रत्यहे याति सममूत्रानुसारतः ।
2,तृतीयं लङ्काक्षितिज- मुन्मण्डलम्11।,तृतीयं लङ्काक्षितिज- मुन्मण्डलम्' ।
3,"( ५६ ओर दौ'र्या मन्दकेन्द्रत्पा तिजाग्नया, सा ...","( ५६ ओर दौ'र्या मन्दकेन्द्रत्पा तिजाग्नया, सा ..."
4,इति पञ्चविंशं सूत्रम् ।,इति पञ्चविंशं सूत्रम् ।
...,...,...
98811,गुरोः षोडशांशाः १६ ।,गुरोः षोडशांशाः १६ ।
98812,तस्मादयथार्थप्रत्यवस्थानात् यत्किञ्चिदेतत् ।,तस्मादयथार्थप्रत्यवस्थानात् यत्किञ्चिदेतत् ।
98813,इति भगवद्गीतोपबृंहणमुपर्युक्त एवार्थेऽनुकूलं द...,इति भगवद्कीतोयृंहणमुपर्युक्त एवार्थऽनुकूलं दृश...
98814,यदि स्वविषयत-ज्जातीयान्यवृत्तिरनैकान्तिक इति म...,यदि स्वविपयत-ज्जातीयान्यवृत्तिरनैकान्तिक इति म...


In [28]:
for data in train_ds:
    if len(list(graphemes(data['sentence'])))!=len(list(graphemes(data['sentence_noisy']))):
        raise ValueError()

In [29]:
sentences, sentences_noisy = preproc_df(dev_df)
dev_df = pd.DataFrame({
    'sentence': sentences,
    'sentence_noisy': sentences_noisy,
})  
dev_ds = datasets.Dataset.from_pandas(dev_df)

  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 5000/5000 [00:00<00:00, 5059.79it/s]


In [30]:
sentences, sentences_noisy = preproc_df(test_df)
test_df = pd.DataFrame({
    'sentence': sentences,
    'sentence_noisy': sentences_noisy,
})  
test_ds = datasets.Dataset.from_pandas(test_df)

100%|██████████| 5000/5000 [00:00<00:00, 5026.57it/s]


In [31]:
char_max_len = 0
char_max = ''
sentence_max = '' 
for sentence, sentence_noisy in zip(sentences,sentences_noisy):
    for char in graphemes(sentence):
        if len(char)>char_max_len:
            char_max_len=len(char)  
            char_max = char 
            sentence_max = sentence

In [32]:
sentence_max, char_max_len, char_max

('तत्र स्वस्वकक्षास्थितलिप्तानां लघुमहत्त्वात् लिप्तावशेन शीघ्रमन्दत्व- मुच्चवशेन च गतीनामुपपन्नम् ।',
 3,
 'नां')

In [33]:
ds = datasets.DatasetDict({
    'train': train_ds,
    'dev': dev_ds,
    'test': test_ds
})

In [34]:
ds.push_to_hub(repo_id=f'jwengr/PeOcrSanskritPreproc', private=False)

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/jwengr/PeOcrSanskritPreproc/commit/1b236f2e390b139ed11f5218418b9b03fd96ac69', commit_message='Upload dataset', commit_description='', oid='1b236f2e390b139ed11f5218418b9b03fd96ac69', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jwengr/PeOcrSanskritPreproc', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jwengr/PeOcrSanskritPreproc'), pr_revision=None, pr_num=None)

In [20]:
len(list(graphemes('ज्योतिष्ठोमेन')))

7